In [ ]:
import pandas as pd
import gzip

In [ ]:
def parse(filename):
  f = gzip.open(filename, 'rb')
  entry = {}
  for l in f:
    l = str(l, 'utf8')
    l = l.strip()
    colonPos = l.find(':')
    if colonPos == -1:
      yield entry
      entry = {}
      continue
    eName = l[:colonPos]
    rest = l[colonPos+2:]
    entry[eName] = rest
  yield entry

def getDF(path):
    i = 0
    data = []
    for d in parse(path):
        data.append(d)
        # i += 1
        # if i >= 1000:
        #     break
    return pd.DataFrame(data)
  
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        pass

    try:
        import unicodedata
        unicodedata.numeric(s)
        return True
    except (TypeError, ValueError):
        pass

    return False

def filte(df: pd.DataFrame, thresh1=20, thresh2=10):
    """Genrate users and items to be filted
    """
    users = set()
    for k, v in df['reviewerID'].value_counts().items():
        if v < thresh1:
            users.add(k)

    items = set()
    for k, v in df['asin'].value_counts().items():
        if v < thresh2:
            items.add(k)

    return users, items

In [ ]:
file_path = '/data/amazon/amazon_book/Books.txt.gz'
book_df = getDF(file_path)

In [ ]:
book_df.drop(['product/title', 'review/profileName', 'review/helpfulness', 'review/score', 'review/summary', 'review/text'], axis=1, inplace=True)
book_df.rename(columns={'review/userId': 'reviewerID', 'product/productId': 'asin', 'product/price': 'price', 'review/time': 'time'}, inplace=True)

In [ ]:
book_df['price'].fillna('unknown', inplace=True)

In [ ]:
book_df.dropna(inplace=True)

In [ ]:
book_df.drop(book_df[book_df['reviewerID'] == 'unknown'].index, inplace=True)

In [ ]:
filte_users, filte_items = filte(book_df)  # users and items to be filted
book_df.drop(book_df[book_df['reviewerID'].isin(filte_users)].index, inplace=True)
book_df.drop(book_df[book_df['asin'].isin(filte_items)].index, inplace=True)

# filted_df = book_df.drop(book_df[book_df['reviewerID'].isin(filte_users)].index)
# filted_df.drop(filted_df[filted_df['asin'].isin(filte_items)].index)

In [ ]:
book_df['price'] = book_df['price'].map(lambda x: 0 if not is_number(x) else float(x))
book_df['price'] = pd.qcut(book_df['price'], q=50, duplicates='drop', labels=False)

In [ ]:
book_df['time'] = book_df['time'].astype('int64')

In [ ]:
book_df

In [ ]:
book_df.to_csv('/data/amazon/amazon_book/amazon_book.txt', index=False)